In [1]:
from filler import fill_redacted_text, evaluate_redacted
import json
import tqdm
import time
import pandas as pd

In [2]:
data = pd.read_csv("inputs/radiology.csv")

data.head()
print(len(data))

1000


In [3]:
notes = data["text"]
notes.head()

0    CT CHEST WITHOUT CONTRAST\n\nREASON FOR EXAM: ...
1    MEDICAL HISTORY:  ___ woman with hemolytic ane...
2    INDICATION:  ___ year old man with metastatic ...
3    INDICATION:  ___ man with right-sided testicul...
4    EXAMINATION:  CT LOWER EXT W/C LEFT\n\nINDICAT...
Name: text, dtype: object

In [4]:
notes = notes.sample(500)
notes

357    EXAMINATION:  CHEST (PORTABLE AP)\n\nINDICATIO...
539    INDICATION:  ___ female with acute mental stat...
379    INDICATION:  Intoxicated patient status post a...
169    EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...
974    EXAMINATION:  CT CHEST W/CONTRAST\n\nINDICATIO...
                             ...                        
267    EXAMINATION:  MRI OF THE PELVIS AND SACRUM, PR...
619    EXAMINATION:  BILATERAL DIGITAL 2D SCREENING M...
127    INDICATION:  Fall on to right side with compla...
662    EXAMINATION:  SHOULDER (AP, NEUTRAL AND AXILLA...
195    INDICATION:  Homeless, smoker with hemoptysis....
Name: text, Length: 500, dtype: object

In [6]:
filled_data = []
total = 0

for note in tqdm.tqdm(notes):
    retries = 0
    while retries < 3:
        try:
            redacted = fill_redacted_text(note)
            evaluate_report = evaluate_redacted(json.dumps(redacted, indent=2))  
            
            if evaluate_report["passed"]:
                filled_text = redacted["mapping_text"]
                redacted_text = redacted["mapping_text"]
                
                for pii_token in redacted["pii_mappings"]:
                    filled_text = filled_text.replace(pii_token, redacted["pii_mappings"][pii_token]["value"])
                    
                filled_data.append({
                    "original_text": note,
                    "filled_text": filled_text,
                    "redacted_text": redacted["mapping_text"],
                    "pii_mappings": redacted["pii_mappings"],
                })
                
                total += 1
                break
            else:
                print(f"Failed to fill the note: {note}")
                print(evaluate_report["reasoning"])
                time.sleep(1)
            
        except Exception as e:
            retries += 1
            print(f"Error: {e}")
            time.sleep(1)

json.dump(filled_data, open("outputs/radiology_filled.json", "w"))
print(f"Generated {total} filled notes")

  0%|          | 0/500 [00:00<?, ?it/s]

Failed to fill the note: EXAMINATION:  CHEST (PORTABLE AP)

INDICATION:  ___ year old woman with catatonia, taking PO for first time today,
developed new oxygen requirement, temp 95.6  // concern for aspiration
pneumonitis or pneumonia

TECHNIQUE:  AP radiograph of the chest.

COMPARISON:  Chest radiograph ___.

IMPRESSION: 

Low lung volumes are noted. There is no focal consolidation, pleural effusion
or pneumothorax. Linear atelectasis in the left lower lobe is unchanged. The
cardiomediastinal silhouette is stable in appearance, again noting a large
hiatal hernia. No acute osseous abnormalities are identified. There are severe
degenerative changes of the bilateral glenohumeral joints.

The mapping_text contains a placeholder '___' in the COMPARISON section that was not replaced with a PII token. Additionally, there appears to be a typo in the COMPARISON section with '<<<' (three angle brackets) instead of '<<' (two angle brackets) for PII_2.


  2%|▏         | 11/500 [00:17<10:13,  1.25s/it]

Failed to fill the note: HISTORY:  Pain.

FINDINGS:  No previous images.  Three views show the bony structures and joint
spaces to be within normal limits with no evidence of joint effusion.

The original text does not contain any "___" placeholders that need to be replaced with PII tokens, and the mapping_text shows no PII tokens (like <<PII_1>>, <<PII_2>>, etc.) present. This appears to be a case where no PII placeholders were identified in the source text, making both completeness and mapping consistency irrelevant as there are no tokens to evaluate.
Failed to fill the note: HISTORY:  Pain.

FINDINGS:  No previous images.  Three views show the bony structures and joint
spaces to be within normal limits with no evidence of joint effusion.

The original text contains no PII placeholders (___) that need replacement, and the mapping_text shows no PII tokens like <<PII_1>> that require entries in pii_mappings. The text appears to be a standard medical report without any PII data to gener

  3%|▎         | 17/500 [00:30<12:39,  1.57s/it]

...

Failed to fill the note: CT CHEST WITHOUT CONTRAST

REASON FOR EXAM:  ___ man with aortic stenosis, CAD, non-Hodgkin
lymphoma, prostate cancer, evaluate aorta calcifications.

TECHNIQUE:  Chest MDCT was performed without contrast using 5-mm and 1.25-mm
axial slice thickness.  Coronal and sagittal reformations were also obtained.

FINDINGS:  No prior study for comparison.

Aortic valvular calcifications are very severe, extending 2 cm upward in the
anterior wall of the ascending aorta.  The rest of the ascending aorta is not
calcified.  Aortic arch and thoracic descending aortic calcifications are
mild-to-moderate.  Coronary artery, mitral annulus, and aortic annulus
calcifications are severe.  Aortic calcifications are also severe at the
origin of the celiac trunk and both renal arteries.

Bilateral axillary lymph nodes are enlarged, up to at 17 x 26 mm on the right
and 19 x 28 mm on the left.  Right subpectoral lymph node is enlarged up to
1.3 cm. Mediastinal lymph nodes are scattered

 99%|█████████▉| 497/500 [13:06<00:03,  1.33s/it]

Failed to fill the note: INDICATION:  Fall on to right side with complaints of proximal right thumb
pain and minimal small snuffbox tenderness.

COMPARISON:  None.

THREE VIEWS OF THE RIGHT WRIST, THREE VIEWS OF THE RIGHT HAND:  No fracture or
dislocation is identified.  Degenerative changes are seen within the wrist
including involvement of the first CMC, triscaphe, and radiocarpal joints
characterized by joint space narrowing, subchondral cystic and sclerotic
changes, and osteophyte formation.  Additionally, degenerative changes are
seen involving the first IP, first MTP, and second through fifth PIP and DIP
joints characterized by joint space narrowing, subchondral cystic and
sclerotic changes, and osteophyte formation.  No radiopaque foreign bodies or
soft tissue calcifications are seen.  No soft tissue gas is present.  No focal
lytic or sclerotic osseous abnormalities are otherwise identified.

IMPRESSION:  No fracture or dislocation.  Osteoarthritis of the hand and wrist
as descr

100%|██████████| 500/500 [13:11<00:00,  1.58s/it]

Generated 500 filled notes
